In [13]:
import numpy as np
import os
import pickle
import torch
from typing import List, Dict
import pandas as pd
import torch.nn.functional as F

In [14]:
dataset = "fmnist"
attack_defense_data = {
    "NEUROTOXIN": {
        "AVG": "../NormalRun/FL_Backdoor_CV/resultWithTime/2024-09-12_23-26-43-avg_NEUROTOXIN-fmnist/",
        # "FLAME": "FL_Backdoor_CV/2024-09-11_01-57-26/",
        "FLTRUST": "FL_Backdoor_CV/2024-09-11_12-39-47/",
    },
    "MR": {
        "AVG": "FL_Backdoor_CV/2024-09-11_16-29-41/",
        # "FLAME": "FL_Backdoor_CV/2024-09-11_05-15-58/",
        "FLTRUST": "FL_Backdoor_CV/2024-09-11_06-21-04/",
    },
    "EDGE_CASE": {
        "AVG": "FL_Backdoor_CV/2024-09-11_17-32-14/",
        # "FLAME": "FL_Backdoor_CV/2024-09-11_10-44-13/",
        "FLTRUST": "FL_Backdoor_CV/2024-09-11_13-45-18/",
    },
}

In [15]:
# for attack, attack2data in attack_defense_data.items():
#     true_labels = []
#     labels_get_by_Euclid = []
#     labels_get_by_manhattan = []
#     labels_get_by_cosine = []
#     labels_get_by_chi_square = []
#     for denfense, data_folder in attack2data.items():
#         if denfense != "AVG":
#             continue
#         participant_file_name = f"{data_folder}participants/participants.csv"
#         participants = np.genfromtxt(
#             participant_file_name, delimiter=",", dtype=None, encoding="utf-8"
#         )
#         participants = participants[1:].T
#         for i in range(len(participants) - 1):
#             file_name = f"{data_folder}model_updates/{dataset}_{attack}_{i}.pkl"
#             if not os.path.exists(file_name):
#                 print(f"File {file_name} not found")
#                 continue
#             with open(file_name, "rb") as file:
#                 model_updates = pickle.load(file)


# roundNum = 10
# dirPath = '../NormalRun/FL_Backdoor_CV/resultWithTime/2024-09-12_23-26-43-avg_NEUROTOXIN-fmnist/'
# pklName = os.path.join(dirPath, f'model_updates/fmnist_NEUROTOXIN_{roundNum}.pkl')
# participantFilePath = os.path.join(dirPath, 'participants/participants.csv')
# # participantFileDF = pd.read_csv(participantFilePath)
# # # 提取参与者数组（第一列）和轮次（从第二列开始的部分）
# # clients = participantFileDF.iloc[1:, 0].tolist()  # 客户端列表
# # roundsArray = participantFileDF.iloc[1:, 1:].to_numpy()  # 轮次数据（50行x30列）
# # print(roundsArray)
# participants = np.genfromtxt(
#     participantFilePath, delimiter=",", dtype=None, encoding="utf-8"
# )
# participants = participants[1:].T
# participants_thisRound = participants[roundNum]
# print(participants_thisRound)

# with open(pklName, 'rb') as f:
#     update: Dict[str, torch.Tensor] = pickle.load(f)
# # print(update)
# # weight0 = update['base_model.model.vision_model.encoder.layers.0.self_attn.k_proj.lora_A.default.weight']
# # weight0.shape  # torch.Size([50, 12288])
# userUpdates = [torch.empty(0) for _ in range(50)]
# for layerKey, values in update.items():
#     # print(layerKey)
#     # print(values)
#     # print(values.shape)  # torch.Size([50, 12288])
#     for i in range(values.shape[0]):
#         trueUser = participants_thisRound[i]
#         userUpdates[trueUser] = torch.cat((userUpdates[trueUser], values[i].cpu()), 0)
# print(userUpdates[0].shape)


def get_all_user_updates(roundNum: int, dirPath: str) -> torch.Tensor:
    # 设置轮次编号和文件路径
    roundNum = 10
    dirPath = '../NormalRun/FL_Backdoor_CV/resultWithTime/2024-09-12_23-26-43-avg-fmnist_NEUROTOXIN/'
    pklPrefix = os.path.basename(os.path.normpath(dirPath)).split('-')[-1]
    pklName = os.path.join(dirPath, f'model_updates/{pklPrefix}_{roundNum}.pkl')
    participantFilePath = os.path.join(dirPath, 'participants/participants.csv')

    # 读取参与者数组
    participants = np.genfromtxt(
        participantFilePath, delimiter=",", dtype=None, encoding="utf-8"
    )
    participants = participants[1:].T
    participants_thisRound = participants[roundNum]  # 获取当前轮次的参与者
    # print(participants_thisRound)

    # 加载 .pkl 文件
    with open(pklName, 'rb') as f:
        update: Dict[str, torch.Tensor] = pickle.load(f)

    # 初始化一个列表来存储每个客户端的更新，使用 GPU 张量
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    userUpdates = [torch.empty((0,), device=device) for _ in range(50)]  # 假设50个客户端

    # 遍历每个层的更新
    for layerKey, values in update.items():
        # 将每个值移到 GPU 上
        values = values.to(device)
        for i in range(values.shape[0]):  # 遍历每个客户端的梯度
            trueUser = participants_thisRound[i]  # 获取当前轮次的客户端ID
            # 如果为空张量
            if userUpdates[trueUser].numel() == 0:
                userUpdates[trueUser] = values[i]  # 直接赋值
            else:
                userUpdates[trueUser] = torch.cat((userUpdates[trueUser], values[i]), 0)  # 在GPU上拼接

    userUpdates = torch.stack(userUpdates)  # 转换为张量
    # # 打印第一个用户的更新形状来检查结果
    # print(userUpdates[0].shape)
    return userUpdates

In [16]:
userUpdates = get_all_user_updates(10, '../NormalRun/FL_Backdoor_CV/resultWithTime/2024-09-12_23-26-43-avg-fmnist_NEUROTOXIN/')
print(userUpdates.shape)
print(userUpdates)

torch.Size([50, 2674688])
tensor([[-0.0176, -0.0016, -0.0176,  ..., -0.0016, -0.0002,  0.0100],
        [-0.0176, -0.0016, -0.0176,  ..., -0.0013, -0.0014,  0.0109],
        [-0.0176, -0.0016, -0.0176,  ..., -0.0007, -0.0016,  0.0093],
        ...,
        [-0.0079, -0.0005, -0.0079,  ..., -0.0012,  0.0008,  0.0040],
        [-0.0079, -0.0005, -0.0079,  ..., -0.0015,  0.0009,  0.0034],
        [-0.0079, -0.0006, -0.0079,  ..., -0.0012, -0.0004,  0.0037]],
       device='cuda:0')


In [17]:
def cosine_similarity_matrix(tensor: torch.Tensor) -> torch.Tensor:
    """
    计算一个二维 tensor 中每两个向量之间的余弦相似度矩阵。

    参数:
    - tensor (torch.Tensor): 形状为 (N, D) 的二维张量，N 是客户端数量，D 是特征维度。

    返回:
    - similarity_matrix (torch.Tensor): 形状为 (N, N) 的张量，每个元素表示两个向量之间的余弦相似度。
    """
    # 归一化每个向量
    normalized_tensor = F.normalize(tensor, p=2, dim=1)
    
    # 计算余弦相似度矩阵
    similarity_matrix = torch.mm(normalized_tensor, normalized_tensor.T)
    
    return similarity_matrix


In [18]:
similarity_matrix = cosine_similarity_matrix(userUpdates)
print(similarity_matrix.shape)
print(similarity_matrix)

torch.Size([50, 50])
tensor([[1.0000, 0.9437, 0.9741,  ..., 0.8990, 0.9020, 0.9061],
        [0.9437, 1.0000, 0.9496,  ..., 0.8677, 0.8740, 0.8812],
        [0.9741, 0.9496, 1.0000,  ..., 0.8939, 0.8934, 0.9036],
        ...,
        [0.8990, 0.8677, 0.8939,  ..., 1.0000, 0.9747, 0.9624],
        [0.9020, 0.8740, 0.8934,  ..., 0.9747, 1.0000, 0.9686],
        [0.9061, 0.8812, 0.9036,  ..., 0.9624, 0.9686, 1.0000]],
       device='cuda:0')
